In [ ]:
# hide
from nbdev.showdoc import *
from nbdev.imports import *
if not os.environ.get("IN_TEST", None):
    assert IN_NOTEBOOK
    assert not IN_COLAB
    assert IN_IPYTHON

In [ ]:
#hide
#skip
! [ -e /content ] && pip install -Uqq fastrl['dev']  # upgrade fastrl on colab

In [ ]:
# default_exp data.block

In [ ]:
# export
# Python native modules
import os
# Third party libs
from fastcore.all import *
from fastai.torch_basics import *
from fastai.data.all import *
from fastai.basics import *
from torch.utils.data import Dataset
from torch import nn


import numpy as np
import gym
import time,sys
import torch.multiprocessing as mp

# Local modules

# Data Block
> Fastrl transforms for iterating through environments

In [ ]:
# export
class DQN(Module):
    def __init__(self):
        self.policy=nn.Sequential(
            nn.Linear(4,50),
            nn.ReLU(),
            nn.Linear(50,2),
            nn.ReLU()
        )
    
    def forward(self,x): 
        return torch.argmax(self.policy(x),dim=0)

In [ ]:
# export
class TestDatasetNoModule(IterableDataset):
    def __init__(self,device='cpu'):
        self.device=device
        self.pids=[os.getpid()]
        
    def __len__(self): return 10
    def __getitem__(self,idx):
        print('starting')
        sys.stdout.flush()
        try:
            print('making env')
            print('pid is: ',os.getpid(),flush=True)
            self.pids.append(os.getpid())
#             env=gym.make('CartPole-v1')
            return torch.rand(1,4).to(device=self.device)
        except Exception as e:
            print(e,'it crashed lol omg')
            sys.stdout.flush()
            return torch.rand(1,4).to(device=self.device)

In [ ]:
for x in DataLoader(TestDatasetNoModule(),num_workers=0):
    print(x)

NotImplementedError: 

In [ ]:
# export

class TestDataset(Dataset):
    def __init__(self,policy,device='cpu'):
        self.policy=policy
        self.device=device
        self.policy.to(device=self.device)
        
        self.pids=mp.Queue()
        self.pids.put(os.getpid())
        self.envs=mp.Queue()
        self.envs.put(os.getpid())

        self.env=gym.make('CartPole-v1')
        
    def __len__(self): return 100
    def __getitem__(self,idx):
        self.pids.put(os.getpid())
        self.envs.put(id(self.env))
        try:
#             env=gym.make('CartPole-v1')
            next_state=self.env.reset()
            print(id(self.env),' ')
            print('pid is: ',os.getpid(),flush=True)
            next_state, r, is_done, _=self.env.step(self.policy(Tensor(next_state).to(device=self.device)).cpu().numpy())
            if is_done:next_state=self.env.reset()
            return Tensor(next_state).to(device=self.device)
        except Exception as e:
            print(e)
            return Tensor(np.random.rand(0,4)).to(device=self.device)

In [ ]:
mp.Queue().empty()

True

In [ ]:
dqn=DQN()
ds=TestDataset(dqn)
for x in DataLoader(ds,num_workers=0):
    print(dqn(x))
    print(x)
while not ds.pids.empty(): print(ds.pids.get(),end=' ')
while not ds.envs.empty(): print(ds.envs.get(),end=' ')
# print(list(ds.envs))

139977286222800  
pid is:  54
tensor(0)
tensor([ 0.0290, -0.2360, -0.0339,  0.2361])
139977286222800  
pid is:  54
tensor(0)
tensor([-0.0256, -0.2272, -0.0222,  0.2737])
139977286222800  
pid is:  54
tensor(0)
tensor([-0.0462, -0.2335,  0.0098,  0.2826])
139977286222800  
pid is:  54
tensor(0)
tensor([-0.0214, -0.1958, -0.0264,  0.2993])
139977286222800  
pid is:  54
tensor(0)
tensor([-0.0428, -0.2287,  0.0270,  0.3159])
139977286222800  
pid is:  54
tensor(0)
tensor([ 0.0307, -0.1540,  0.0202,  0.2689])
139977286222800  
pid is:  54
tensor(0)
tensor([-0.0435, -0.2423, -0.0322,  0.3182])
139977286222800  
pid is:  54
tensor(0)
tensor([-0.0360, -0.1623,  0.0246,  0.2944])
139977286222800  
pid is:  54
tensor(0)
tensor([-0.0426, -0.1676, -0.0286,  0.2643])
139977286222800  
pid is:  54
tensor(0)
tensor([ 0.0416, -0.1694, -0.0244,  0.2964])
139977286222800  
pid is:  54
tensor(0)
tensor([-0.0456, -0.1895, -0.0380,  0.2938])
139977286222800  
pid is:  54
tensor(0)
tensor([ 0.0221, -0.1848,

In [ ]:
# hide
from nbdev.export import *
from nbdev.export2html import *
from nbdev.cli import make_readme
make_readme()
notebook2script()
notebook2html()

converting /home/fastrl_user/fastrl/nbs/index.ipynb to README.md
Converted 00_nbdev_extension.ipynb.
Converted 05_data.block.ipynb.
Converted 05_data.test_async.ipynb.
Converted 20_test_utils.ipynb.
Converted index.ipynb.
Converted nbdev_template.ipynb.
converting: /home/fastrl_user/fastrl/nbs/05_data.block.ipynb
